# Model Değerlendirmesi: PSNR ve SSIM

Eğitilen modelin kalitesini ölçmek için iki metrik kullanıyoruz:

- **PSNR (Peak Signal-to-Noise Ratio)**: Yüksek değer = daha az gürültü
- **SSIM (Structural Similarity Index)**: 1'e yakın = yapısal benzerlik yüksek

---

## Önkoşullar

- `03_training.ipynb` ile eğitilmiş model (.h5)
- Validation dataset (NPY formatında)

In [ ]:
# Google Colab için
from google.colab import drive
drive.mount('/content/drive')

## 1. Model Mimarisini Yükle

In [ ]:
%run 01_model_architecture.ipynb

## 2. Kaydedilmiş Modeli Yükle

In [ ]:
import os
import glob
import numpy as np
import tensorflow as tf
from tqdm.notebook import tqdm

# Model checkpoint dizini
checkpoint_dir = "/content/drive/MyDrive/Tasarım Dersi/Projects/model_checkpoints"

# En son kaydedilen modeli bul
list_of_files = glob.glob(os.path.join(checkpoint_dir, 'G_epoch_*.h5'))
if not list_of_files:
    raise ValueError("Model dosyası bulunamadı!")

latest_file = max(list_of_files, key=os.path.getctime)
print(f"Yüklenen Model: {os.path.basename(latest_file)}")

# Generator'ı yükle
generator = build_generator()
generator.load_weights(latest_file)
print("✅ Model hazır.")

## 3. Validation Verisini Yükle

Tüm veriyi RAM'e alıyoruz (yaklaşık 40-50 saniye sürer).

In [ ]:
# Veri yolu
dataset_path = "/content/drive/MyDrive/Tasarım Dersi/Projects/data/processed_data_npy"

# Dosyaları al
all_files_A = sorted(glob.glob(os.path.join(dataset_path, 'trainA') + '/*.npy'))
all_files_B = sorted(glob.glob(os.path.join(dataset_path, 'trainB') + '/*.npy'))

# Sadece eşleşenleri al
filenames_A = {os.path.basename(f) for f in all_files_A}
filenames_B = {os.path.basename(f) for f in all_files_B}
common_filenames = sorted(list(filenames_A.intersection(filenames_B)))

# %10 validation için ayır
from sklearn.model_selection import train_test_split
filtered_files_A = [os.path.join(dataset_path, 'trainA', f) for f in common_filenames]
filtered_files_B = [os.path.join(dataset_path, 'trainB', f) for f in common_filenames]

_, val_files_A, _, val_files_B = train_test_split(
    filtered_files_A, filtered_files_B, test_size=0.10, random_state=42
)

print(f"Validation görüntü sayısı: {len(val_files_A)}")

In [ ]:
print("\nVeriler RAM'e yükleniyor...")

all_inputs = []
all_targets = []

for fA, fB in tqdm(zip(val_files_A, val_files_B), total=len(val_files_A)):
    try:
        imgA = np.load(fA)
        imgB = np.load(fB)

        # Boyut düzeltme
        if imgA.ndim == 2: imgA = np.expand_dims(imgA, axis=-1)
        if imgB.ndim == 2: imgB = np.expand_dims(imgB, axis=-1)

        all_inputs.append(imgA)
        all_targets.append(imgB)
    except:
        continue

X_val = np.array(all_inputs)
Y_val = np.array(all_targets)
print(f"✅ Veri Hazır. Toplam Görüntü: {len(X_val)}")

## 4. PSNR ve SSIM Hesapla

RAM'i korumak için 50'şerli batch'ler halinde hesaplıyoruz.

In [ ]:
print("\nAnaliz Başlıyor...")

psnr_scores = []
ssim_scores = []

batch_size = 50
total_samples = len(X_val)

for i in tqdm(range(0, total_samples, batch_size)):
    batch_input = X_val[i : i + batch_size]
    batch_target = Y_val[i : i + batch_size]

    # Tahmin
    batch_pred = generator(batch_input, training=False)

    # PSNR/SSIM hesapla
    # Veriler -1 ile 1 arasında olduğu için max_val=2.0
    batch_psnr = tf.image.psnr(batch_target, batch_pred, max_val=2.0).numpy()
    batch_ssim = tf.image.ssim(batch_target, batch_pred, max_val=2.0).numpy()

    psnr_scores.extend(batch_psnr)
    ssim_scores.extend(batch_ssim)

## 5. Sonuçlar

In [ ]:
avg_psnr = np.mean(psnr_scores)
std_psnr = np.std(psnr_scores)
avg_ssim = np.mean(ssim_scores)
std_ssim = np.std(ssim_scores)

print("\n" + "="*40)
print("📊 FİNAL SONUÇLAR")
print("="*40)
print(f"Ortalama PSNR: {avg_psnr:.4f} dB (±{std_psnr:.4f})")
print(f"Ortalama SSIM: {avg_ssim:.4f} (±{std_ssim:.4f})")
print("="*40)